In [1]:
from roboflow import Roboflow

In [2]:
import csv

In [3]:
import json
cred_path = r".secrets/credentials.json"

In [4]:
with open(cred_path, "r") as f:
    credentials = json.load(f)

In [5]:
rf = Roboflow(api_key=credentials["api_key"])

In [6]:
project = rf.workspace().project(credentials["model_id"])

loading Roboflow workspace...
loading Roboflow project...


In [7]:
model = project.version(1).model

In [8]:
import cv2

In [9]:
def read_text(image_path):
    response = model.predict(image_path, confidence=40, overlap=30).json()
    sorted_predictions = sorted(response['predictions'], key=lambda pred: pred['x'])
    val = ''
    for pred in sorted_predictions:
        val = val + pred['class']
    num_val = float(val)
    weight = f"{num_val:.5f}"
    return weight

In [10]:
def save_to_csv(data, csv_path='weights.csv'):
    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Product Name", "Detected Weight"])
        writer.writerows(data)

In [22]:
def capture_frames_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    captured_frames = []
    frame_number = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow('Video', frame)
        key = cv2.waitKey(1) & 0xFF

        if (key == ord(' ')):  
            captured_frames.append(frame)
            print(f"Captured frame {frame_number}")

        elif (key == ord('q')):  
            break

        frame_number += 1

    cap.release()
    cv2.destroyAllWindows()
    return captured_frames

In [18]:
def save_frame_as_image(frame, image_path):
    cv2.imwrite(image_path, frame)

In [26]:
video_path = input("Enter the path to the video file: ")
num_products = int(input("Enter the number of products: "))

products = []

for _ in range(num_products):
    product_name = input(f"Enter the product name for product {_ + 1}: ")
    products.append(product_name)

csv_path = input("Enter the CSV file path to save the data (default is 'weights.csv'): ")

if not csv_path:
    csv_path = "weights.csv"

data = []

frames = capture_frames_from_video(video_path)

for i, frame in enumerate(frames):
    image_path = f"Frames Captured\\frame_{i}.jpg"
    save_frame_as_image(frame, image_path)
    
    product_index = i % num_products  
    product_name = products[product_index]
    
    weight = read_text(image_path)
    data.append([product_name, weight])
    print(f"Product: {product_name}, Weight: {weight}")

save_to_csv(data, csv_path)

print(f"Data saved to {csv_path}")

Captured frame 170
Captured frame 354
Captured frame 560
Product: A, Weight: 11903.00000
Product: B, Weight: 3662.00000
Product: C, Weight: 35226.70000
Data saved to weights.csv
